In [2]:
from pycocotools.coco import COCO
import os
import glob
import shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import tqdm
import pandas as pd
dataset = COCO('annotations/manual_val.json')


loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [ ]:

# move all images into one folder
img_src_dir = ""
img_dir = "images"
image_sub_folders = [
   "upper_00000_00500", "upper_00500_01000", "upper_01000_01500", "upper_01500_02000",
   "upper_02000_03000", "upper_03000_04000", "upper_04000_05000", "upper_05000_06000",
   "upper_06000_07000", "upper_07000_08000", "upper_08000_09000", "upper_09000_10000"
   ]
os.mkdir(img_dir)
for image_sub_folder in image_sub_folders:
  sub_folder_path = os.path.join(img_src_dir, image_sub_folder)
  for file_path in glob.glob(os.path.join(sub_folder_path, '*.*')):
    shutil.move(file_path, img_dir)
  os.rmdir(sub_folder_path)

In [5]:
category_ids = dataset.getCatIds()
annotation_ids = dataset.getImgIds()
SHOW = False

def get_ball_poly(annotation):
  for ann in annotation:
    segmentations = ann['segmentation']
    category_id = ann['category_id']
    if category_id == 2:
      return np.array(segmentations[0]).reshape(-1, 2).round().astype(int)

def compute_center(pts):
    x = pts[:, 0]
    y = pts[:, 1]

    A = np.vstack([x, y, np.ones(len(pts))]).T
    B = np.square(x) + np.square(y)

    h, k, _ = np.linalg.lstsq(A, B, rcond=None)[0]/2

    return np.array([h,k])

data = list()

for idx in tqdm.tqdm(annotation_ids):
    img_data = dataset.loadImgs(int(idx))[0]
    img_path = os.path.join(img_dir, img_data["file_name"])
    assert(os.path.isfile(img_path)), img_data["file_name"] 
    img = cv2.imread(img_path)

    pts = get_ball_poly(dataset.loadAnns(dataset.getAnnIds(idx)))

    if pts is None:
       data.append({'image': img_data["file_name"], 'frame_contains_ball': False, 'center_x': None, 'center_y': None, 'radius': None})
    else:
      com = compute_center(pts)
      radius = np.linalg.norm(pts - com, axis=1).mean()
      data.append({'image': img_data["file_name"], 'frame_contains_ball': True, 'center_x': com[0], 'center_y': com[1], 'radius': radius})

    if SHOW:
      fig, ax = plt.subplots()
      circle = Circle(com, radius, color='red', fill=True, alpha=0.5)
      ax.add_patch(circle)
      ax.imshow(img)
      plt.show()

df = pd.DataFrame(data)

df.to_csv("manual_val.csv")


100%|██████████| 151/151 [00:02<00:00, 61.92it/s]
